In [26]:
import csv
import re
from ftfy import fix_text
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
stopw = set(stopwords.words('english'))
from pyresparser import ResumeParser
import os
from docx import Document
from skills_extraction import skills_extractor
import fitz 
import pandas as pd

In [27]:
d_df = pd.read_csv('updated_job_dataset2.csv')

csv_file_path = 'updated_job_dataset2.csv'

In [28]:
file_path = r"C:/Users/vinay/Desktop/MajorProject/resume_analyzer_notebook/VINAYAKA KAMATH.pdf"
skills = []
skills.append(' '.join(word for word in skills_extractor(file_path)))
print(skills)

['algorithms English Python analysis C C++ HTML Java SQL JS programming TECHNICAL JavaScript communication system']


In [29]:
def preprocess_skills(skills):
   
    skills = skills.lower()
   
    skills = re.sub(r'[^a-z0-9\s\+\#]', '', skills)
    
    processed_skills = skills.split()
    return processed_skills

In [30]:

skill_category_mapping = {
    'c++': 'Software',
    'java': 'Software',
    'python': 'Software',
    'html': 'Software',
    'javascript': 'Software',
    'sql': 'Data',
    'system': 'IT',
    'technical': 'IT',
    'english': 'Communication',
    'algorithms': 'programming',
    'analysis': 'analysis',
    'js': 'Software',  
    'programming': 'programming',
    'testing': 'testing',
    'teaching': 'teaching',
    'support': 'support',
    'management': 'management',
    'engineering': 'engineering',
    'research': 'research',
    'design': 'design',
    'consulting': 'consulting',
    'marketing': 'marketing',
    'sales': 'sales',
    'finance': 'finance',
    'accounting': 'accounting',
    'health': 'health',
    'nursing': 'nursing',
    'administration': 'administration',
    'legal': 'legal',
    'writing': 'writing',
    'editing': 'editing',
    'product': 'product',
    'project': 'project',
    'operations': 'operations',
    'customer': 'customer',
    'human resources': 'human resources',
    'it': 'IT',
    'software': 'software',
    'hardware': 'hardware',
    'network': 'network',
    'security': 'security',
    'data': 'data',
    'business': 'business',
    'logistics': 'logistics',
    'supply chain': 'supply chain',
    'construction': 'construction',
    'real estate': 'real estate',
    'manufacturing': 'manufacturing',
    'quality': 'quality',
    'training': 'training',
    'education': 'education',
    'media': 'media',
    'communications': 'Communications',
    'assistant': 'assistant',
    'trainee': 'trainee'
}


In [31]:
def match_skills_to_categories(processed_skills, skill_category_mapping):
    matched_categories = set()
    for skill in processed_skills:
        if skill in skill_category_mapping:
            matched_categories.add(skill_category_mapping[skill])
    return list(matched_categories)

In [32]:
skills_string = ' '.join(skills)


processed_skills = preprocess_skills(skills_string)
print(f"Processed skills: {processed_skills}")


Processed skills: ['algorithms', 'english', 'python', 'analysis', 'c', 'c++', 'html', 'java', 'sql', 'js', 'programming', 'technical', 'javascript', 'communication', 'system']


In [33]:
matched_categories = match_skills_to_categories(processed_skills, skill_category_mapping)
print(f"Matched categories: {matched_categories}")

Matched categories: ['IT', 'Communication', 'analysis', 'Data', 'Software', 'programming']


In [34]:
def create_table(matched_rows):
    if not matched_rows:
        return pd.DataFrame()
    else:
        return pd.DataFrame(matched_rows)


In [35]:
def find_matching_row(csv_file, matched_categories):
    matched_rows = []
    with open(csv_file, 'r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['Keyword'] in matched_categories:
                matched_rows.append(row)
    return matched_rows

In [36]:
matched_rows = find_matching_row(csv_file_path, matched_categories)

In [37]:
table_df = create_table(matched_rows)
table_df.head(1000)

,Title,Company,Salary,Description,city,url,Processed_JD,Experience,Keyword
0,Data Entry,Futurism,,Research and obtain further information for in...,India,https://in.indeed.com/rc/clk?jk=a1dc42307340ca...,,0,Data
1,Data Entry Intern,AppSierra Solutions Private Limited,,Collaborate with team members to address data-...,India,https://in.indeed.com/rc/clk?jk=bb135b45c43311...,,0,Data
2,Data Entry Operator/Computer Operator,Dight International Chambers,,The Data Entry Operator will be responsible fo...,"Amaravati, Andhra Pradesh",https://in.indeed.com/rc/clk?jk=420c2a57d51924...,,0,Data
3,Data Entry Operator,Banasuri Enterprises,,Communication Skills- Verbal And Written. Accu...,"Bengaluru, Karnataka",https://in.indeed.com/rc/clk?jk=98b1e726d40d4f...,,0,Data
4,Data Scientist,Ventura Securities,,Empowering you to enjoy your “Me-time” with th...,Hybrid work in India,https://in.indeed.com/rc/clk?jk=2506d2cda39498...,,0,Data
...,...,...,...,...,...,...,...,...,...
451,Android Apps Developer / Software Developer,Unifylabs Systems,,"Position ID : AD-SD-002. Skills : Android, Eco...","Bengaluru, Karnataka",https://in.indeed.com/rc/clk?jk=dcbc815f673368...,android,0,Software
452,Software Engineer II Android,ACV Auctions,,If you are looking for a career at a dynamic c...,"Chennai, Tamil Nadu",https://in.indeed.com/rc/clk?jk=89d541c09bafb6...,android,0,Software
453,"Senior Software Engineer, Android",Google,,Bachelor’s degree or equivalent practical expe...,"Bengaluru, Karnataka",https://in.indeed.com/rc/clk?jk=3dcc89e51f2cdf...,android,0,Software
454,Senior Software Engineer - Android,Intuit,,We are looking for team members that love new ...,"Bengaluru, Karnataka",https://in.indeed.com/rc/clk?jk=c9e66b33e3d774...,android,0,Software
